# 概要
- テストデータに類似している分子を学習に使える様に類似度を評価する

In [1]:
import os
from pathlib import Path

def is_kaggle_kernel():
    return os.path.exists('/kaggle/working')

if is_kaggle_kernel():

    BASE_DIR = Path("/kaggle")
    DATA_DIR = BASE_DIR / "input"
    OUTPUT_DIR = BASE_DIR / "working"
    print('on kaggle notebook')

else:
    BASE_DIR = Path(os.getcwd()) / './../'
    DATA_DIR = BASE_DIR / "data"
    OUTPUT_DIR = BASE_DIR / "output/eda"
    
class paths:    
    DATA_DIR = DATA_DIR
    TRAIN_PATH = DATA_DIR / "train.parquet"
    TEST_PATH = DATA_DIR / "test.parquet"
    OUTPUT_DIR = OUTPUT_DIR
    SHRUNKEN_DATA_DIR = DATA_DIR / "shrunken-train-set"
    
    OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

In [2]:
import os
import gc
import math
import numpy as np
import pandas as pd
from glob import glob
# import duckdb
# import lightgbm as lgb
from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import Draw

import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from rdkit.Chem.Scaffolds import MurckoScaffold
from sklearn.neighbors import NearestNeighbors
import pickle

In [3]:
# mainのmoleculeのECFP4
df_train_main_ecfp4 = np.load(paths.DATA_DIR / "leash-bio-processed-dataset/train.ecfp4.packed.npz", allow_pickle=True)['ecfp']
df_test_main_ecfp4 = np.load(paths.DATA_DIR / "leash-bio-processed-dataset/test.ecfp4.packed.npz", allow_pickle=True)['ecfp']

In [21]:
# メモリに乗らないのでランダムにサンプリングする
sampling_num = int(len(df_test_main_ecfp4)/800)
df_test_main_ecfp4_small = df_test_main_ecfp4[np.random.choice(df_test_main_ecfp4.shape[0], sampling_num, replace=False)]
test_ecfp4 = np.unpackbits(df_test_main_ecfp4_small, axis=1)

In [23]:
n_neighbors = int(test_ecfp4.shape[0] / 100)
ad_model = NearestNeighbors(n_neighbors=n_neighbors, metric='euclidean')  # AD モデルの宣言
ad_model.fit(test_ecfp4)

NearestNeighbors(metric='euclidean', n_neighbors=10)

In [24]:
chunk_size = 100000
n_chunks = len(df_train_main_ecfp4) // chunk_size + 1

distances = np.array([])
for i in tqdm(range(n_chunks)):
# for i in tqdm(range(5)):
    start = i * chunk_size
    end = (i + 1) * chunk_size
    
    train_ecfp4_chunk = np.unpackbits(df_train_main_ecfp4[start:end, :], axis=1)
    d = ad_model.kneighbors(train_ecfp4_chunk, n_neighbors=n_neighbors)[0].mean(axis=1)
    
    distances = np.concatenate([distances, d])

 36%|███▌      | 356/985 [14:15<25:11,  2.40s/it]


KeyboardInterrupt: 

In [42]:
# from rdkit.Chem import QED
from rdkit.Chem.Descriptors3D import Volume
from rdkit.Chem.Descriptors import qed

mol = Chem.MolFromSmiles('CCO')
qed(mol)

ImportError: cannot import name 'Volume' from 'rdkit.Chem.Descriptors3D' (/usr/local/lib/python3.10/site-packages/rdkit/Chem/Descriptors3D.py)

In [28]:
# distanceを読み込む
distances = np.load(paths.DATA_DIR / "my-data/distance_to_test_data.npy")
distances.shape

(35600000,)